In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ---------------------------------------- 2.4/2.4 GB ? eta 0:00:00
     ---------------------------------------- 4.8/4.8 MB 7.9 MB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 5.4 MB/s eta 0:00:00


In [2]:
!pip install beautifulsoup4 transformers requests

     -------------------------------------- 128.2/128.2 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 908.1 kB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 1.2 MB/s eta 0:00:00
     ------------------------------------ 267.8/267.8 kB 531.0 kB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 1.1 MB/s eta 0:00:00
     ------------------------------------ 182.4/182.4 kB 916.0 kB/s eta 0:00:00
     ------------------------------------ 151.6/151.6 kB 822.1 kB/s eta 0:00:00


In [9]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import torch 
import re

In [10]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading: 100%|██████████| 39.0/39.0 [00:00<00:00, 4.90kB/s]
c:\Users\Pranav\anaconda3\envs\tensor\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pranav\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 953/953 [00:00<00:00, 118kB/s]
Downloadin

In [52]:
tokens = tokenizer.encode("Detective Batman at its peak! Great storyline. Just as dark a universe as we've come to expect from DC. The gloomy, gritty, dark tone of this film is exactly what I wanted. When you think the movie is over, there's more. Beautiful cinematography. Great score.", return_tensors='pt')
result = model(tokens)
sentiment = int(torch.argmax(result.logits)) + 1
sentiment

5

In [39]:
r = requests.get('https://www.imdb.com/title/tt1877830/reviews?ref_=tt_urv')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*text show-more__control.*')
results = soup.find_all( 'div',{'class': regex})
reviews = [result.text for result in results]

In [42]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])
df

,reviews
0,Detective Batman at its peak! Great storyline....
1,I just got out of The BatmanThis movie really ...
2,I have been absolutely fizzing to see 'The Bat...
3,"The Riddler(Paul Dano, spot-on. How did it tak..."
4,STAR RATING: ***** Brilliant **** Very Good **...
5,Everything about this movie is trying too hard...
6,I'm not sure where to begin as I don't general...
7,"Matt Reeves The Batman is a hell of a movie, i..."
8,The headline can be taken two ways. If you rem...
9,Wasn't a fan of Emo Batman. Sorry Robert Patti...


In [55]:
def sentiment_scorer(review):
    tokens = tokenizer.encode(str(review), return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [57]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_scorer(x[:512]))
df

,reviews,sentiment
0,Detective Batman at its peak! Great storyline....,5
1,I just got out of The BatmanThis movie really ...,2
2,I have been absolutely fizzing to see 'The Bat...,2
3,"The Riddler(Paul Dano, spot-on. How did it tak...",2
4,STAR RATING: ***** Brilliant **** Very Good **...,4
5,Everything about this movie is trying too hard...,2
6,I'm not sure where to begin as I don't general...,3
7,"Matt Reeves The Batman is a hell of a movie, i...",5
8,The headline can be taken two ways. If you rem...,4
9,Wasn't a fan of Emo Batman. Sorry Robert Patti...,2
